In [1]:
import os
import sys
import json
from anthropic import Anthropic, AnthropicError

notebook_path = os.getcwd()
project_path = os.path.abspath(os.path.join(notebook_path, '..','..'))
print(f"Project path: {project_path}")
# Add the project path to sys.path
sys.path.append(project_path)

Project path: c:\Users\dalej\Documents\_Coding\DragonRegen


## Testing Action Generator

In [4]:
s_test_json = """{"users": {     "purpose": "Stores information about registered users/customers",     "fields": {       "user_id": "unique ID, primary key for user identification",       "username": "unique username for login",       "email": "unique email address for user contact",       "password_hash": "securely stored password hash",       "first_name": "user's first name",       "last_name": "user's last name",       "phone_number": "contact phone number",       "created_at": "timestamp when user account was created",       "updated_at": "timestamp when user account was last updated"     }   },   "addresses": {     "purpose": "Stores shipping and billing addresses for users",     "fields": {       "address_id": "unique ID, primary key for address identification",       "user_id": "foreign key referencing users table",       "address_type": "type of address (shipping/billing)",       "street_address": "street name and number",       "city": "city name",       "state": "state or province",       "postal_code": "zip or postal code",       "country": "country name",       "is_default": "boolean indicating if this is the default address for the user"     }   },   "products": {     "purpose": "Stores product information available for sale",     "fields": {       "product_id": "unique ID, primary key for product identification",       "name": "product name",       "description": "detailed product description",       "price": "current product price",       "category_id": "foreign key referencing categories table",       "inventory_count": "current available inventory",       "image_url": "URL to product image",       "created_at": "timestamp when product was added",       "updated_at": "timestamp when product was last updated"     }   },   "categories": {     "purpose": "Stores product categories for organization",     "fields": {       "category_id": "unique ID, primary key for category identification",       "name": "category name",       "description": "category description",       "parent_category_id": "self-referencing foreign key for hierarchical categories"     }   },   "orders": {     "purpose": "Stores customer orders",     "fields": {       "order_id": "unique ID, primary key for order identification",       "user_id": "foreign key referencing users table",       "order_date": "date and time when order was placed",       "total_amount": "total order amount",       "shipping_address_id": "foreign key referencing addresses table",       "billing_address_id": "foreign key referencing addresses table",       "status": "current order status (pending, shipped, delivered, etc.)",       "payment_method": "method used for payment"     }   },   "order_items": {     "purpose": "Stores individual items within an order",     "columns": {       "order_item_id": "unique ID, primary key for order item identification",       "order_id": "foreign key referencing orders table",       "product_id": "foreign key referencing products table",       "quantity": "quantity of product ordered",       "unit_price": "price of product at time of purchase",       "subtotal": "total price for this line item (quantity * unit_price)"     }   },   "payments": {     "purpose": "Stores payment information for orders",     "fields": {       "payment_id": "unique ID, primary key for payment identification",       "order_id": "foreign key referencing orders table",       "payment_date": "date and time of payment",       "payment_method": "method of payment (credit card, PayPal, etc.)",       "amount": "payment amount",       "status": "payment status (pending, completed, failed, etc.)",       "transaction_id": "external payment processor transaction ID"     }   },   "cart_items": {     "purpose": "Stores items in user shopping carts",     "fields": {       "cart_item_id": "unique ID, primary key for cart item identification",       "user_id": "foreign key referencing users table",       "product_id": "foreign key referencing products table",       "quantity": "quantity of product in cart",       "added_at": "timestamp when item was added to cart"     }   },   "reviews": {     "purpose": "Stores product reviews from users",     "columns": {       "review_id": "unique ID, primary key for review identification",       "product_id": "foreign key referencing products table",       "user_id": "foreign key referencing users table",       "rating": "numerical rating (e.g., 1-5 stars)",       "comment": "text review content",       "created_at": "timestamp when review was created"     }   } }"""
test_json = json.loads(s_test_json)
for key in test_json.keys():
    print(key) 

users
addresses
products
categories
orders
order_items
payments
cart_items
reviews


In [5]:
schema_json = {key: val for key, val in test_json.items() if key in ['addresses','products', 'categories']}
print(schema_json.keys())
print(schema_json)

dict_keys(['addresses', 'products', 'categories'])
{'addresses': {'purpose': 'Stores shipping and billing addresses for users', 'fields': {'address_id': 'unique ID, primary key for address identification', 'user_id': 'foreign key referencing users table', 'address_type': 'type of address (shipping/billing)', 'street_address': 'street name and number', 'city': 'city name', 'state': 'state or province', 'postal_code': 'zip or postal code', 'country': 'country name', 'is_default': 'boolean indicating if this is the default address for the user'}}, 'products': {'purpose': 'Stores product information available for sale', 'fields': {'product_id': 'unique ID, primary key for product identification', 'name': 'product name', 'description': 'detailed product description', 'price': 'current product price', 'category_id': 'foreign key referencing categories table', 'inventory_count': 'current available inventory', 'image_url': 'URL to product image', 'created_at': 'timestamp when product was added

In [6]:
from src.AIGuardian.ActionGenerator.SchemaRefiner import SchemaRefiner

test_schema_refiner = SchemaRefiner(schema_json)
print(test_schema_refiner)


In [7]:
test_schema_refiner.run(user_prompt="")

create_params: {'max_tokens': 10000, 'temperature': 0.1, 'stop_sequences': ['</JSON_Template>'], 'model': 'claude-3-7-sonnet-20250219', 'messages': [{'role': 'user', 'content': 'Please take the following <column_information> to choose what type of column a column most likely is.\n<column_information>\nTable Purpose: Stores shipping and billing addresses for users\nColumn Info: "address_id": "unique ID, primary key for address identification"\n</column_information>\n\nChoose one column_type from this list <choices> below, structure as column_type: description of what qualifies a column as that type.\n<choices>\nColBasic: A back stop column for your basic data types with no other identifying attributes.\nStringBasic: This is a basic string column generator that can be used to generate a column of strings with a specified length and format. It is not intended for free text or complex string generation.\nCategorical: a column that represents a category or type, often with a limited set of 

AttributeError: 'dict' object has no attribute 'append'

In [8]:
print(len(test_schema_refiner.child_action))
print(test_schema_refiner.child_action[0].output_params)


3
{'addresses': {'purpose': 'Stores shipping and billing addresses for users', 'fields': [{'name': 'address_id', 'type': 'Integer', 'nullable': False, 'metadata': {'description': 'unique ID, primary key for address identification', 'unique_fl': True, 'default_value': None, 'col_type': 'Unique_Identifier'}}, {'name': 'user_id', 'type': 'Integer', 'nullable': False, 'metadata': {'description': 'foreign key referencing users table', 'unique_fl': False, 'default_value': None, 'col_type': 'Unique_Identifier'}}, {'name': 'address_type', 'type': 'String', 'nullable': False, 'metadata': {'description': 'Type of address that indicates whether it is a shipping or billing address for a user', 'unique_fl': False, 'column_values': ['shipping', 'billing'], 'col_type': 'Categorical'}}, {'name': 'street_address', 'type': 'String', 'nullable': False, 'metadata': {'description': 'street name and number', 'unique_fl': False, 'default_value': None, 'col_type': 'Text'}}, {'name': 'city', 'type': 'String', 

In [ ]:
d_tables = {}
for action in test_schema_refiner.child_action:
    if isinstance(action, ColumnRefiner):
        d_tables = {**d_tables, **action.output_params}
        # d_tables[table_name] = action.output_params

## Test Table Column Action Generator

In [3]:
table_json = test_json['users']
table_json['table_name'] = 'users'
table_json['fields'] = table_json.pop('columns')
print(table_json)

{'purpose': 'Stores information about registered users/customers', 'table_name': 'users', 'fields': {'user_id': 'unique ID, primary key for user identification', 'username': 'unique username for login', 'email': 'unique email address for user contact', 'password_hash': 'securely stored password hash', 'first_name': "user's first name", 'last_name': "user's last name", 'phone_number': 'contact phone number', 'created_at': 'timestamp when user account was created', 'updated_at': 'timestamp when user account was last updated'}}


In [4]:
from src.AIGuardian.ActionGenerator.ColumnRefiner import ColumnRefiner

test_col_refiner = ColumnRefiner(input_params=table_json)
print(test_col_refiner)

In [5]:
test_col_refiner.run(user_prompt="")


create_params: {'max_tokens': 10000, 'temperature': 0.1, 'stop_sequences': ['</JSON_Template>'], 'model': 'claude-3-7-sonnet-20250219', 'messages': [{'role': 'user', 'content': 'Please take the following <column_information> to choose what type of column a column most likely is.\n<column_information>\nTable Purpose: Stores information about registered users/customers\nColumn Info: "user_id": "unique ID, primary key for user identification"\n</column_information>\n\nChoose one column_type from this list <choices> below, structure as column_type: description of what qualifies a column as that type.\n<choices>\nColBasic: A back stop column for your basic data types with no other identifying attributes.\nStringBasic: This is a basic string column generator that can be used to generate a column of strings with a specified length and format. It is not intended for free text or complex string generation.\nCategorical: a column that represents a category or type, often with a limited set of va

In [6]:
print(json.dumps(test_col_refiner.output_params, indent=2))

{
  "users": {
    "purpose": "Stores information about registered users/customers",
    "fields": [
      {
        "name": "user_id",
        "type": "Integer",
        "nullable": false,
        "metadata": {
          "description": "Unique ID, primary key for user identification in the registered users/customers table.",
          "unique_fl": true,
          "primary_key": true,
          "default_value": null,
          "col_type": "Unique_Identifier"
        }
      },
      {
        "name": "username",
        "type": "String",
        "nullable": false,
        "metadata": {
          "description": "Unique username for login to the system. Used as the primary identifier for user authentication.",
          "unique_fl": true,
          "default_value": null,
          "col_type": "Unique_Identifier"
        }
      },
      {
        "name": "email",
        "type": "String",
        "nullable": false,
        "metadata": {
          "description": "unique email address for 

## Testing the Column Registry

In [3]:
from src.DataCreator.ColGenerators.ColGenRegistry import ColGenResistry

ColGenResistry.get_all_descriptions()


{'ColBasic': 'A back stop column for your basic data types with no other identifying attributes.',
 'StringBasic': 'This is a basic string column generator that can be used to generate a column of strings with a specified length and format. It is not intended for free text or complex string generation.',
 'Categorical': 'a column that represents a category or type, often with a limited set of values.',
 'StringFirstName': 'A column that stores the first name of a person.',
 'StringLastName': 'A column that stores the Last name of a person.'}

In [8]:
import json

s_st = """--> The Beginning of the ColGenResistry.get_all_descriptions() output
"""+ '\n'.join([str(key)+': '+str(val) for key, val in ColGenResistry.get_all_descriptions().items()])+"""
--> THE END
"""
               
print(s_st)

--> The Beginning of the ColGenResistry.get_all_descriptions() output
ColBasic: A back stop column for your basic data types with no other identifying attributes.
StringBasic: This is a basic string column generator that can be used to generate a column of strings with a specified length and format. It is not intended for free text or complex string generation.
Categorical: a column that represents a category or type, often with a limited set of values.
StringFirstName: A column that stores the first name of a person.
StringLastName: A column that stores the Last name of a person.
--> THE END



In [5]:
o_colgen = ColGenResistry.get_col_generator('ColBasic')
print(o_colgen)
print(o_colgen.get_metadata_json())
print(o_colgen.get_examples())

<class 'src.DataCreator.ColGenerators.ColBasic.ColBasic'>
{'description': 'Place the description of the column here.', 'unique_fl': True, 'default_value': None}
Example 1:
        Purpose: "This table is used to store user information."
        Column Info: "user_id": "unique ID representing each user."
        Output:
        <JSON_Template>
        {"name": "user_id", "type": "Integer", "nullable": False, 
            "metadata": {"description": "unique ID representing each user.", 
            "unique_fl": True,
            "default_value": None}}
        </JSON_Template>


### Data Column Type

In [2]:
from src.AIGuardian.ActionsFrame.DataColumnType import DataColumnType

# Inputs
action_parameters = {
                "table_name": 'members',
                "purpose": "To track all the information about members of the gym.",
                "column_name": 'membership',
                "description": 'The kind of membership the member has.',
            }

test_type_action = DataColumnType(input_params=action_parameters)
print(test_type_action)

In [3]:
test_type_action.run(user_prompt="")
print(test_type_action.response)

create_params: {'max_tokens': 10000, 'temperature': 0.1, 'stop_sequences': ['</JSON_Template>'], 'model': 'claude-3-7-sonnet-20250219', 'messages': [{'role': 'user', 'content': '\nRespond in JSON format like this:\n{\n    "choice": "selected_option",\n    "reason": "brief explanation"\n}\n\n<Examples>\nExample 1:\n<column_information>\nTable Purpose: Stores information about gym members.\nColumn Info: "member_name": "The first name of the gym member."\n</column_information>\nOutput:\n<JSON_Template>\n{\n    "choice": "First_Name",\n    "reason": "This represents the first name of a person, which is a common attribute in member records."\n}\n</JSON_Template>\n\nExample 2:\n<column_information>\nTable Purpose: Stores information about gym members.\nColumn Info: "subscription": "The type of subscription the member has."\n</column_information>\nOutput:\n<JSON_Template>\n{{\n    "choice": "Categorical",\n    "reason": "There is a finite number of options for memberships at a gym."\n}}\n</JS

In [4]:
print(test_type_action.output_params)

{'reason': 'There is a finite number of options for memberships at a gym.', 'col_type': 'Categorical'}


### Data Column Refiner

In [5]:
from src.AIGuardian.ActionsFrame.DataColumnRefiner import DataColumnRefiner

# Inputs
action_parameters = {
                "table_name": 'members',
                "purpose": "To track all the information about members of the gym.",
                "column_name": 'membership',
                "description": 'The kind of membership the member has.',
            }

test_action = DataColumnRefiner(input_params=action_parameters, parent_action=test_type_action)
print(test_action)

In [6]:
test_action.run(user_prompt="")
print(test_action.response)


create_params: {'max_tokens': 10000, 'temperature': 0.1, 'stop_sequences': ['</JSON_Template>'], 'model': 'claude-3-7-sonnet-20250219', 'messages': [{'role': 'user', 'content': 'Please take the following <column_information> to fill in JSON values and add additional metadata for the column.\n        <column_information>\n        Table Purpose: To track all the information about members of the gym.\n        Column Info: "membership": "The kind of membership the member has."\n        </column_information>\n        Please provide the response in a JSON format like the <JSON_Template> below. If the JSON value has (Optional) in the value you may or may not included.\n        There are examples below at <Examples>.\n        <JSON_Template>\n        {"name": "example_column", "type": "Integer", "nullable": true, "metadata": {"description": "Place the description of the column here. Add detail if current description is vague.", "unique_fl": "Choose True or False", "default_value": "(Optional) 

## Test Data Struct

In [ ]:
# from src.AIGuardian.ActionsFrame.DataStructCreate import DataStructCreate

# datastruct_action = DataStructCreate()
# prompt = """Create a data structure for a gym membership system."""
# datastruct_action.run(prompt)
# print(datastruct_action.response)

SQL: INSERT INTO ai_actions_log (action_name, action_version, parent_action_id, group_action_id, description, sequence_number, created_dt, updated_dt, status, metadata) 
        OUTPUT INSERTED.action_id
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        
create_params: {'max_tokens': 10000, 'temperature': 0.1, 'stop_sequences': ['</JSON_Template>'], 'model': 'claude-3-7-sonnet-20250219', 'messages': [{'role': 'user', 'content': 'Please take the following <business_requirements> and create the table list and schema for the the request.\n        <business_requirements>\n        Create a data structure for a gym membership system.\n        </business_requirements>\n        Please provide the response in a JSON format like the <JSON_Template> below:\n        <JSON_Template>\n        {"table_name_1": {\n            "purpose": "short description of tables usage in the architecture",\n            "fields":{"column_name_1": "unique ID, short description of column",\n            , "column_n